# Ensemble learning

## Mục tiêu
- Sử dụng các mô hình đã xây dựng ở các notebook mục 1 → 8 (ví dụ: Linear, Ridge, Lasso, các mô hình tree-based, v.v.) làm các mô hình cơ sở (base learners),
- Tổ hợp các mô hình cơ sở bằng các phương pháp ensemble: Voting (trung bình), Stacking (meta-learner), Bagging/Boosting nếu cần,
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`,
- Tối ưu hóa siêu tham số cho các thành phần khi cần bằng 5-Fold Cross-Validation,
- Đánh giá mô hình ensemble trên train/validation/test và báo các metric: RMSE, MAE, R2",
- Lưu các mô hình cơ sở và mô hình ensemble đã huấn luyện"

## Giới thiệu ngắn về Ensemble Learning - Voting
- Kết hợp dự đoán của nhiều mô hình bằng cách trung bình (cho bài toán regression) hoặc bỏ phiếu đa số (cho classification).

## Bước 1 - Import các thư viện cần thiết


### 1.1. Import thư viện

In [35]:
from pathlib import Path
import joblib
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

### 1.2. Cấu hình thư mục

In [11]:
RANDOM_STATE = 42
os.makedirs("../model/ensemple_learning", exist_ok=True)

## Bước 2 - Đọc dữ liệu đã tiền xử lý
- Đọc `../data/processed/test.csv` 
- In thông tin cơ bản (shape, head)

In [18]:
# Đọc dữ liệu
test_df = pd.read_csv('../data/processed/test.csv')
validation_df = pd.read_csv('../data/processed/val.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {test_df.shape}")
print(f"Kích thước tập validation: {validation_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập test (đã được chuẩn hóa):")
test_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (1085, 15)
Kích thước tập validation: (1085, 15)

5 dòng đầu tiên của tập test (đã được chuẩn hóa):


,country_name,country_code,year,population,poverty_ratio,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,slum_population,labor_force
0,Ethiopia,ETH,2011,0.450767,0.944960,0.935901,60.595,-0.624530,1.329134,-1.641488,-2.092241,-3.048195,-0.526097,1.969506,2.009406
1,North Macedonia,MKD,2024,-0.231712,-0.407597,-1.972767,72.603,-0.258277,-0.121702,-1.366208,0.590208,0.453709,-0.269510,-0.911910,-0.851642
2,Curacao,CUW,2024,-0.243936,-0.591325,-0.743950,72.603,-0.414833,0.007849,-0.197441,0.590208,0.453709,-0.269510,-0.434261,-0.005686
3,Brazil,BRA,2004,1.119556,-0.030314,-0.099813,71.361,-0.488893,0.395738,-0.467033,0.509135,0.439114,-0.302740,0.159824,0.628914
4,Ecuador,ECU,2006,-0.139872,-0.233328,0.255778,74.010,-0.506054,0.151321,-0.357408,0.491511,-0.038560,-0.285603,1.301866,0.686304


## Bước 3 - Chuẩn bị dữ liệu cho mô hình
- Xác định feature_cols (loại bỏ `life_expectancy`, `country_name`, `country_code`)
- Tách X_test, y_test

In [25]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in test_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_val = validation_df[feature_cols]
y_val = validation_df['life_expectancy']
X_test = test_df[feature_cols]
y_test = test_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 12

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. poverty_ratio
  4. pop_growth
  5. gdp_per_capita
  6. gdp_growth
  7. sanitation
  8. electricity
  9. water_access
  10. co2_emissions
  11. slum_population
  12. labor_force

Kích thước X_train: (1085, 12)
Kích thước y_train: (1085,)


## Bước 4 - Load các mô hình cơ sở (base learners)


In [21]:
BASE_DIR = Path("../model")

# ===== 1. Linear / Ridge / Lasso =====
linear_reg = joblib.load(BASE_DIR / "1_linear_regression" / "linear.pkl")
ridge_reg  = joblib.load(BASE_DIR / "1_linear_regression" / "ridge.pkl")
lasso_reg  = joblib.load(BASE_DIR / "1_linear_regression" / "lasso.pkl")

# ===== 2. SVM Regression =====
linear_svr = joblib.load(BASE_DIR / "2_SVM_regression" / "linear_svr.pkl")
svr_rbf    = joblib.load(BASE_DIR / "2_SVM_regression" / "svr_rbf.pkl")

# ===== 3. Decision Tree =====
decision_tree = joblib.load(BASE_DIR / "3_decision_tree" / "decision_tree.pkl")

# ===== 4. Random Forest =====
random_forest = joblib.load(BASE_DIR / "4_random_forest" / "random_forest.pkl.gz")

# ===== 5. GBM =====
gbm_model = joblib.load(BASE_DIR / "5_GBM" / "GBM.pkl.gz")

# ===== 6. LightGBM =====
lightgbm_model = joblib.load(BASE_DIR / "6_lightgbm" / "lightgbm.pkl")

# ===== 7. XGBoost =====
# xgboost_model = joblib.load(BASE_DIR / "7_xgboost" / "xgboost.pkl.gz")

# ===== 8. CatBoost =====
catboost_model = joblib.load(BASE_DIR / "8_catboost" / "catboost.pkl.gz")

In [22]:
# Gom vào dict để dùng cho ensemble
base_models = {
    "linear": linear_reg,
    "ridge": ridge_reg,
    "lasso": lasso_reg,
    "linear_svr": linear_svr,
    "svr_rbf": svr_rbf,
    "decision_tree": decision_tree,
    "random_forest": random_forest,
    "gbm": gbm_model,
    "lightgbm": lightgbm_model,
    # "xgboost": xgboost_model,
    "catboost": catboost_model,
}

# Bước 6 - Xây dựng mô hình ensemble
- Định nghĩa Voting Regressor (Voting có trọng số theo RMSE (model tốt hơn → weight cao hơn))
- Vì các base model đều được train sẵn, do đó ko cần khai train lại từ đầu

### 6.1. Tính RMSE trên validation cho từng base model

In [29]:
def rmse_for_base_models(models, X, y):
    """Hàm tính RMSE cho các base models."""
    rmse_results = {}
    for name, model in models.items():
        y_pred = model.predict(X)
        rmse = root_mean_squared_error(y_true=y, y_pred=y_pred)
        rmse_results[name] = rmse
    return rmse_results

rmse = rmse_for_base_models(base_models, X_val, y_val)
print("RMSE của các base models trên tập validation:")
for name, value in rmse.items():
    print(f"  {name}: {value:.4f}")

RMSE của các base models trên tập validation:
  linear: 4.0341
  ridge: 4.0331
  lasso: 4.0311
  linear_svr: 4.1461
  svr_rbf: 2.7625
  decision_tree: 2.5494
  random_forest: 1.3083
  gbm: 1.4199
  lightgbm: 1.2948
  catboost: 1.1594


### 6.2. Trọng số của các model

In [31]:
inv_rmse = {name: 1.0 / rmse for name, rmse in rmse.items()}
total_inv = sum(inv_rmse.values())
weights = {name: val / total_inv for name, val in inv_rmse.items()}

print("\nTrọng số:")
for name, w in weights.items():
    print(f"{name}: {w:.3f}")


Trọng số:
linear: 0.051
ridge: 0.051
lasso: 0.051
linear_svr: 0.050
svr_rbf: 0.075
decision_tree: 0.081
random_forest: 0.158
gbm: 0.145
lightgbm: 0.159
catboost: 0.178


### 6.3. Hàm predict với weighted voting

In [ ]:
def weighted_voting_predict(X, model_dict=base_models, model_weights=weights):
    y_pred = np.zeros(X.shape[0], dtype=float)
    for name, w in model_weights.items():
        model = model_dict[name]
        y_pred += w * model.predict(X)
    return y_pred



## 7. Đánh giá trên tập Test


### 7.1. Đánh giá từng base model

In [36]:
def evaluation(X_test, model):
    y_pred = model.predict(X_test)
    rmse = root_mean_squared_error(y_true=y_test, y_pred=y_pred)
    mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
    r2 = r2_score(y_true=y_test, y_pred=y_pred)
    return rmse, mae, r2

In [37]:
for name, model in base_models.items():
    rmse_val, mae_val, r2_val = evaluation(X_test, model)
    print(f"{name} - RMSE: {rmse_val:.4f}, MAE: {mae_val:.4f}, R2: {r2_val:.4f}")

linear - RMSE: 4.0163, MAE: 3.0430, R2: 0.7817
ridge - RMSE: 4.0172, MAE: 3.0440, R2: 0.7816
lasso - RMSE: 4.0166, MAE: 3.0435, R2: 0.7817
linear_svr - RMSE: 4.1692, MAE: 3.0782, R2: 0.7648
svr_rbf - RMSE: 2.9396, MAE: 1.9954, R2: 0.8831
decision_tree - RMSE: 2.5930, MAE: 1.8583, R2: 0.9090
random_forest - RMSE: 1.4801, MAE: 0.8717, R2: 0.9704
gbm - RMSE: 1.5137, MAE: 0.9380, R2: 0.9690
lightgbm - RMSE: 1.5402, MAE: 0.9346, R2: 0.9679
catboost - RMSE: 1.3301, MAE: 0.8021, R2: 0.9761


In [34]:
y_pred = weighted_voting_predict(X_test, base_models, weights)
rmse_ensemble = root_mean_squared_error(y_true=y_test, y_pred=y_pred)
mae_ensemble = mean_absolute_error(y_true=y_test, y_pred=y_pred)
r2_ensemble = r2_score(y_true=y_test, y_pred=y_pred)

print("RMSE: ", rmse_ensemble)
print("MAE: ", mae_ensemble) 
print("R2: ", r2_ensemble)

RMSE:  1.8183289705937196
MAE:  1.2479657712692827
R2:  0.955254362629285
